In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.neighbors import NearestNeighbors 
from scipy.sparse import csr_matrix
import csv

In [3]:
book = pd.read_csv('../input/bookcrossing-dataset/Book reviews/Book reviews/BX_Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user = pd.read_csv('../input/bookcrossing-dataset/Book reviews/Book reviews/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
rating = pd.read_csv('../input/bookcrossing-dataset/Book reviews/Book reviews/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")

In [4]:
book.columns = ['ISBN', 'BookTitle', 'BookAuthor', 'YearOfPublication', 'Publisher', 'ImageUrlS', 'ImageUrlM', 'ImageUrlL']
user.columns = ['UserID', 'Location', 'Age']
rating.columns = ['UserID', 'ISBN', 'BookRating'] 

In [5]:
book

In [6]:
user

In [7]:
rating

In [8]:
c1_usid = rating['UserID'].value_counts()  
c1_usid

In [9]:
rating = rating[rating['UserID'].isin(c1_usid[c1_usid >= 10].index)]
rating['UserID'].value_counts()

In [10]:
c2_usid = rating['BookRating'].value_counts()  
c2_usid

In [11]:
rating = rating[rating['BookRating'].isin(c2_usid[c2_usid >= 10].index)]
c1 = rating['BookRating'].value_counts()
c1

In [12]:
book_rating = pd.merge(rating, book, on='ISBN')
book_rating

In [13]:
columns = ['BookAuthor', 'YearOfPublication', 'Publisher', 'ImageUrlS', 'ImageUrlM', 'ImageUrlL']
book_rating = book_rating.drop(columns, axis=1)
book_rating

In [14]:
book_rating = book_rating.dropna()
book_rating

In [15]:
bkrating_cnt = (book_rating.groupby(by = ['BookTitle'])['BookRating'].count().reset_index().rename(
columns = {'BookRating': 'TotalRating'})[['BookTitle', 'TotalRating']]) 
bkrating_cnt

In [16]:
rating_bkrating_cnt = book_rating.merge(bkrating_cnt, left_on = 'BookTitle', right_on = 'BookTitle', how = 'left')
rating_bkrating_cnt

In [17]:
a = rating_bkrating_cnt['TotalRating']
a[a>=10].value_counts()

In [18]:
a[a>=10]

In [19]:
b=rating_bkrating_cnt[rating_bkrating_cnt['TotalRating']>=10]
b

In [20]:
user.isna().sum()

In [21]:
user.drop('Age',axis=1,inplace=True)
user.drop('Location',axis=1,inplace=True)
user

In [22]:
final = b.merge(user,left_on = 'UserID', right_on = 'UserID', how = 'left')
final

In [23]:
final = final.drop_duplicates(['UserID', 'BookTitle'])
final

In [24]:
final.to_csv('Final.csv')

In [25]:
final_pivot = final.pivot(index = 'BookTitle', columns = 'UserID', values = 'BookRating').fillna(0)
final_matrix = csr_matrix(final_pivot.values)

In [26]:
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(final_matrix)

In [27]:
finalmodel = 'Final_Model.sav'
pickle.dump(model, open(finalmodel, 'wb'))

In [28]:
final_pivot

In [29]:
final_matrix

In [31]:
final_pivot.shape[0]

In [40]:
distances, suggestions = model.kneighbors(final_pivot.iloc[237, :].values.reshape(1, -1),n_neighbors = 5)

In [41]:
for i in range(len(suggestions)):
    print(final_pivot.index[suggestions[i]])

In [42]:
index = np.random.choice(final_pivot.shape[0])
for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations :\n'.format(final_pivot.index[index]))
        else:
            print('{0}: {1}'.format(i, final_pivot.index[suggestions.flatten()[i]],distances.flatten()[i]))